# Box Office Mojo Web Scrape and Data Cleaning

### This file will gather data from https://www.boxofficemojo.com and accomplish the following
##### - Grabs all domestic level data for the years 2015-2020
    Example:
    https://www.boxofficemojo.com/year/2015/?sortDir=asc&grossesOption=totalGrosses&sort=releaseDate
##### - Create a dataframe from the data grabbed above
##### - For each movie grabbed above, grab movie specific data by navigating to the movie specific page on the site
    Example:
    https://www.boxofficemojo.com/release/rl947553793/?ref_=bo_yld_table_1
##### - Join the movie specific data grabbed above with our main dataframe
##### - Clean and format our dataframe to the point that EDA can be performed
##### - Export our dataframe as a csv so that we can perform EDA in a fresh jupyter notebook and do not have to run anymore requests

In [ ]:
# Import the library that will be needed in this notebook
import requests
from bs4 import BeautifulSoup 
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import time
import datetime as dt
%matplotlib inline

In [2]:
# The purpose of this dictionary will be used later as a parameter to pass into the requests.get() function
# This should help make my cpu seem more legit in a four loop that will make 4031 requests to the mojo website
heads =  {
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9", 
    "Accept-Encoding": "gzip, deflate, br", 
    "Accept-Language": "en-US,en;q=0.9", 
    "Host": "httpbin.org", 
    "Referer": "https://www.scraperapi.com/blog/5-tips-for-web-scraping/", 
    "Sec-Fetch-Dest": "document", 
    "Sec-Fetch-Mode": "navigate", 
    "Sec-Fetch-Site": "none", 
    "Upgrade-Insecure-Requests": "1", 
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.125 Safari/537.36", 
    "X-Amzn-Trace-Id": "Root=1-5f3a8c4e-59d257c4391e32fa489cf12a"
  }

In [3]:
# Get 2015-2020 movie data from boxofficemojo
# store each soup in a list of soup objects
    # use str.format to update url for each call

soup_array = []
for year in range(2015, 2021):
    url = 'https://www.boxofficemojo.com/year/{}/?sortDir=asc&sort=releaseDate&grossesOption=totalGrosses'.format(str(year))
    time.sleep(np.random.randint(0,5))
    page = requests.get(url)
    soup_array.append(BeautifulSoup(page.content, 'html.parser'))

In [8]:
# These are the html class strings needed to grab the associated data values
columns_to_html_class_guide = [
    ('Title', 'a-text-left mojo-field-type-release mojo-cell-wide'),
    ('Gross', 'a-text-right mojo-field-type-money mojo-estimatable'),
    ('Max_Theaters', 'a-text-right mojo-field-type-positive_integer'),
    ('Opening_Weekend_Gross', 'a-text-right mojo-field-type-money'),
    ('Opening_Weekend_Percent_of_Gross', 'a-text-right mojo-field-type-percent'),
    ('Open_Date', 'a-text-left mojo-field-type-date mojo-sort-column a-nowrap'),
    ('Distributor', 'a-text-left mojo-field-type-studio'),
    ('Year', None)
]

In [9]:
#initialize an empty dictionary. Each Key/value pair is a list, where each specific value corresponds to a single movie
movies = {
    'Title': [],
    'Gross': [],
    'Max_Theaters': [],
    'Opening_Weekend_Theaters': [],
    'Opening_Weekend_Gross': [],
    'Opening_Weekend_Percent_of_Gross' : [],
    'Open_Date': [],
    'Distributor': [],
    'Year': []
}


In [11]:
# This dictionary will help me apply the appropriate year to each movie row down below...Value represents a 
# count of movies in that year from this website's data
movies_per_year = {
    '2015': 0,
    '2016': 0,
    '2017': 0,
    '2018': 0,
    '2019': 0,
    '2020': 0
}


In [12]:
# loop through each year, and for each year, do the following
    # keep track of a movie count, which will be added to the movies_per_year dict
        # note i need to divide movie count by len(columns) at the end of each loop
        # otherwise, i would have too many values by a factor of the len(columns)
    # for each year, iterate through my columns_to_html_class_guide
        # each iteration of the loop will handle a specific column of a data, for example, the title column
    # for each column iteration, iterate through my soup array 
        # for each item in the soup array, i pass in the appropriate index value as well as the appropriate 
        # class parameter. The class parameter is the html string found in the second item of each 
        # tuple in columns_to_html_class_guide
        

for soup_index, year in enumerate(range(2015,2021)):
    movie_count = 0
    for column in columns_to_html_class_guide:
        for item in soup_array[soup_index].find_all('td', class_=column[1]):
            movies[column[0]] = movies[column[0]] + [item.text]
            movie_count += 1
    movies_per_year[str(year)] = movie_count / len(columns_to_html_class_guide) # 


In [13]:
# This will be an array with the same length as the total number of movies we pulled
# Each item in the array will be the href link to the associated movie_details page
href_array = []

In [14]:
# This cell will run through our soup array and populate the href value for each row into the href array

for soup in soup_array:
    for item in soup.find_all('td', columns_to_html_class_guide[0][1]):
        href_array.append(str(item.find_all("a")).split('href=')[1].split(">")[0].strip('\"'))

In [18]:
# This is to initialize empty lists for the values we plan to pull from the movie details page on the mojo website
genres = []
rates = []
runtimes = []

In [19]:
# This cell takes around 20-30 minutes to run. It loops through our href array, and for each value, makes a request
# To the mojo website with the href appended to the end of the URL so that we get the appropriate movie's page
# The for loop at the bottom checks the html of each page for a genre, a runtime, and a rating value. Not all
# movie pages have all these values present. Because of this, we also attach the enumerated index value from our href
# foor loop. This will come into play later when we need to join this data in a pandas dataframe that scraped all the 
# movies from the domestic movies page in mojo

for index, href in enumerate(href_array):

    href_url = 'https://www.boxofficemojo.com{}'.format(href)
    time.sleep(np.random.randint(0,2)/100)    
    href_page = requests.get(href_url, headers=heads)
    href_soup = BeautifulSoup(href_page.content, 'html.parser')
    href_items = href_soup.find_all('div', class_='a-section a-spacing-none')

    for item in href_items:
        spans = item.find_all("span")
        if spans[0].text == 'Genres':
            genres.append((index, spans[1].text.split()))
        if spans[0].text == 'Running Time':
            runtimes.append((index, spans[1].text))
        if spans[0].text.strip() == 'MPAA':
            rates.append((index, spans[1].text))
        

In [77]:
## GET THESE INTO NP ARRAYS, WILL JOIN WITH OUR DF DATAFRAME LATER ON
genres = np.array(genres)
rates = np.array(rates)
runtimes = np.array(runtimes)

In [28]:
# NEED TO ADD ALL ODD INDEXES OF MAX_THEATERS TO OPENING_WEEKEND_THEATERS
    # basically, the html to pull 'Max Theaters' and 'Opening # Theaters' is the same html class structure
    # this was the easiest / first way i thought to easily distinguish the two
    # each time soup is called, it pulled first a value for max theater (comes first in the html code) and then a 
    # value for the opening_#_theaters...so, can easily split by even/odd indexing
new_max = []
new_open = []
for i, item in enumerate(movies['Max_Theaters']):
    if i % 2 == 0:
        new_max.append(item)
    else:
        new_open.append(item)

In [29]:
# This cell takes what we did above and assigns the proper values as seen on the website to max theaters and opening
# weekend theaters
movies['Max_Theaters'] = new_max
movies['Opening_Weekend_Theaters'] = new_open

In [30]:
# This cell will populate the appropriate year into our dictionary of movies. It uses the movies_per_year variable
# determined in our initial forloop to get the movie data (not the for loop to get the genre, rating, runtime data!)
# This cell utilizes a simple if / elif / else clause to get out appropriate years into the dictionary

for i in range(0, len(movies['Title'])):
    if i < movies_per_year['2015']:
        movies['Year'] = movies['Year'] + ['2015']
    elif i < movies_per_year['2016']:
        movies['Year'] = movies['Year'] + ['2016']
    elif i < movies_per_year['2017']:
        movies['Year'] = movies['Year'] + ['2017']
    elif i < movies_per_year['2018']:
        movies['Year'] = movies['Year'] + ['2018']
    elif i < movies_per_year['2019']:
        movies['Year'] = movies['Year'] + ['2019']
    else:
        movies['Year'] = movies['Year'] + ['2020']


In [293]:
# verify all items in dictionary have same length 
for key, value in movies.items():
    print('{}: {} items'.format(key, len(value)))

Title: 4031 items
Gross: 4031 items
Max_Theaters: 4031 items
Opening_Weekend_Theaters: 4031 items
Opening_Weekend_Gross: 4031 items
Opening_Weekend_Percent_of_Gross: 4031 items
Open_Date: 4031 items
Distributor: 4031 items
Year: 4031 items


In [333]:
# CREATE OUR DATAFRAME 
df = pd.DataFrame(movies)

In [335]:
# ADD GENRES
genres_df = pd.DataFrame(genres, columns=['index','Genres']).set_index('index')
df = df.merge(genres_df, how='left', left_on = df.index, right_on = genres_df.index)


In [336]:
# ADD RATINGS
rating_df = pd.DataFrame(rates, columns = ['index', 'Rating'])
rating_df.loc[:,'index'] = rating_df['index'].astype('int')
rating_df.set_index('index', inplace=True)
print(rating_df.index)
print(df.index)
df = df.join(rating_df)

Int64Index([   0,    5,    8,   10,   15,   16,   18,   21,   23,   25,
            ...
            4014, 4016, 4017, 4018, 4019, 4020, 4021, 4023, 4028, 4029],
           dtype='int64', name='index', length=1899)
Int64Index([   0,    1,    2,    3,    4,    5,    6,    7,    8,    9,
            ...
            4021, 4022, 4023, 4024, 4025, 4026, 4027, 4028, 4029, 4030],
           dtype='int64', length=4031)


In [337]:
# ADD RUNTIMES
runtimes_df = pd.DataFrame(runtimes, columns=['index','Runtime'])
runtimes_df.loc[:,'index'] = runtimes_df['index'].astype('int')
runtimes_df.set_index('index', inplace=True)
df = df.join(runtimes_df)

In [338]:
# Clean runtimes
# Convert the object values to numerical values in minutes
def runtimes_minutes(runtime):
    time = 0
    try:
        runtime = runtime.split()
        time = 60*int(runtime[0]) + int(runtime[2])
    except:
        time = 0
    return time

# ------------------------------------------------------------ function ended

df.loc[:, 'Runtime'] = df['Runtime'].apply(runtimes_minutes)

In [339]:
df.head()

,key_0,Title,Gross,Max_Theaters,Opening_Weekend_Theaters,Opening_Weekend_Gross,Opening_Weekend_Percent_of_Gross,Open_Date,Distributor,Year,Genres,Rating,Runtime
0,0,The Woman in Black 2: Angel of Death,"$26,501,323","2,602","2,602","$15,027,415",56.7%,Jan 2,Relativity Media\n\n,2015,"[Drama, Horror, Thriller]",PG-13,98
1,1,The Search for General Tso,"$54,275",13,2,"$9,336",17.2%,Jan 2,IFC Films\n\n,2015,"[Comedy, Documentary, History, Mystery]",NaN,71
2,2,The Taking of Tiger Mountain,"$228,984",13,7,"$50,621",22.1%,Jan 2,Well Go USA Entertainment\n\n,2015,"[Action, Adventure, Biography, History, War]",NaN,141
3,3,Li'l Quinquin,"$7,192",2,1,"$3,204",44.5%,Jan 2,Kino Lorber\n\n,2015,"[Comedy, Crime, Mystery]",NaN,206
4,4,Trace Amounts,"$80,735",7,-,-,-,Jan 2,GathrFilms\n\n,2015,"[Adventure, Documentary, Drama, History]",NaN,97


In [340]:
# Convert Gross column to float
df.loc[:,'Gross'] = df['Gross'].apply(lambda x: float(x.strip('$').replace(',','')))

In [341]:
#Clean Opening_Weekend_Gross column

#replace '-'s w/ str(zero)
df.loc[:, 'Opening_Weekend_Gross'] = df['Opening_Weekend_Gross'].apply(lambda x: '0' if '-' in x else x) 
#convert to floats
df.loc[:,'Opening_Weekend_Gross'] = df['Opening_Weekend_Gross'].apply(lambda x: float(x.strip('$').replace(',','')))


In [342]:
# same process as above. for theater columns, replace the '-' with a string of zero. then, replace all
# values as ints (cant have have a theater!)

df.loc[:, 'Max_Theaters'] = df['Max_Theaters'].apply(lambda x: '0' if '-' in x else x) 
df.loc[:, 'Opening_Weekend_Theaters'] = df['Opening_Weekend_Theaters'].apply(lambda x: '0' if '-' in x else x) 

df.loc[:,'Max_Theaters'] = df['Max_Theaters'].apply(lambda x: int(x.strip('$').replace(',','')))
df.loc[:,'Opening_Weekend_Theaters'] = df['Opening_Weekend_Theaters'].apply(lambda x: int(x.strip('$').replace(',','')))


In [343]:
#convert the opening_weekend_percent_of_gross to a float. 
# Will need to replace '-' with string zeros first. 
# then, will need to strip the '%' value
# lastly, will need to strip the '<' values for the '<0.1' example
df.loc[:, 'Opening_Weekend_Percent_of_Gross'] = df['Opening_Weekend_Percent_of_Gross'].apply(
                                                lambda x: '0' if '-' in x else x) 
df.loc[:, 'Opening_Weekend_Percent_of_Gross'] = df['Opening_Weekend_Percent_of_Gross'].apply(
                                                lambda x: x.strip('%').strip('<'))
df.loc[:, 'Opening_Weekend_Percent_of_Gross'] = df['Opening_Weekend_Percent_of_Gross'].astype('float')

In [344]:
# Now, condence the Year and the Open_Date into one column, then make sure its a dt.date object
    # THIS METHOD WILL BE CALLED BELOW FOR OUR NEW DATE COLUMN 

def to_date(value):
    months = {
                'Jan': 1, 'Feb': 2, 'Mar': 3, 'Apr': 4, 'May': 5, 'Jun': 6, 'Jul': 7, 'Aug': 8, 'Sep': 9, 
                'Oct': 10, 'Nov': 11, 'Dec': 12
                }
    value = value.split()
    return dt.date(int(value[2]), months[value[0]], int(value[1]))

In [345]:
# make new column then call our method to clean it into a better type format

df['Date'] = df['Open_Date'] + ' ' + df['Year']
df.loc[:, 'Date'] = df['Date'].apply(to_date).astype('datetime64[ns]')

In [346]:
# Now, drop the open_date and the year columns as we no longer need them
df.drop(['Open_Date', 'Year'], axis=1, inplace=True)

In [347]:
# We have two values of '\n\n' at the end of each row in Distributor. Let's drop those, then we are done cleaning
df.loc[:, 'Distributor'] = df['Distributor'].apply(lambda x: x.rstrip('\n\n'))

In [349]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4031 entries, 0 to 4030
Data columns (total 12 columns):
 #   Column                            Non-Null Count  Dtype         
---  ------                            --------------  -----         
 0   key_0                             4031 non-null   int64         
 1   Title                             4031 non-null   object        
 2   Gross                             4031 non-null   float64       
 3   Max_Theaters                      4031 non-null   int64         
 4   Opening_Weekend_Theaters          4031 non-null   int64         
 5   Opening_Weekend_Gross             4031 non-null   float64       
 6   Opening_Weekend_Percent_of_Gross  4031 non-null   float64       
 7   Distributor                       4031 non-null   object        
 8   Genres                            3985 non-null   object        
 9   Rating                            1899 non-null   object        
 10  Runtime                           4031 non-null 

In [350]:
df.drop('key_0', axis=1, inplace=True)

In [354]:
df.to_csv('Box_Office_Mojo_Domestic_Box_Office_Chart.csv')

In [355]:
df

,Title,Gross,Max_Theaters,Opening_Weekend_Theaters,Opening_Weekend_Gross,Opening_Weekend_Percent_of_Gross,Distributor,Genres,Rating,Runtime,Date
0,The Woman in Black 2: Angel of Death,26501323.0,2602,2602,15027415.0,56.7,Relativity Media,"[Drama, Horror, Thriller]",PG-13,98,2015-01-02
1,The Search for General Tso,54275.0,13,2,9336.0,17.2,IFC Films,"[Comedy, Documentary, History, Mystery]",NaN,71,2015-01-02
2,The Taking of Tiger Mountain,228984.0,13,7,50621.0,22.1,Well Go USA Entertainment,"[Action, Adventure, Biography, History, War]",NaN,141,2015-01-02
3,Li'l Quinquin,7192.0,2,1,3204.0,44.5,Kino Lorber,"[Comedy, Crime, Mystery]",NaN,206,2015-01-02
4,Trace Amounts,80735.0,7,0,0.0,0.0,GathrFilms,"[Adventure, Documentary, Drama, History]",NaN,97,2015-01-02
...,...,...,...,...,...,...,...,...,...,...,...
4026,The Tax Collector,430423.0,138,129,309694.0,72.0,RLJE Films,"[Action, Crime, Drama, Thriller]",NaN,95,2020-08-07
4027,Peninsula,118647.0,48,48,118647.0,100.0,Well Go USA Entertainment,"[Action, Horror, Thriller]",NaN,116,2020-08-07
4028,Made in Italy,73332.0,111,111,33998.0,46.4,IFC Films,[Comedy],R,94,2020-08-07
4029,The SpongeBob Movie: Sponge On The Run,900000.0,300,300,900000.0,0.0,Paramount Pictures,"[Adventure, Animation, Comedy, Family]",PG,91,2020-08-14
